In [1]:
import torch

# Ciclo de trabajo en un problema de NLP

Hay muchas formas de entrenar una red que se usa para NLP. Nada de esto está muy estandarizado, así que voy a suponer que se trata de un problema de ML normal y olvidarme de la eficiencia computacional por un rato. 

Resumen del ciclo de trabajo:

* Nuestro dataset es conjunto de muestras, cada una de ellas conformada por una secuencia de texto y un label. Además, se dispone de un vocabulario, de manera que todas las palabras de la secuencia de texto de la muestra se pueden mapear a un índice en ese vocabulario (incluído los caracteres especiales para denotar "fin de párrafo", "palabra desconocida", "padding token", etc.). Todo esto se va a definir en una subclase de la clase `torch.utils.data.Dataset`, en donde se sobreescriben los métodos `__len__` y `__getitem__`.

* Una vez definido el dataset, se obtiene el dataloader, el cual es una subclase de la clase `torch.utils.data.DataLoader` y recibe como argumentos una función para muestrear, el tamaño del batch y la instancia de `torch.utils.data.Dataset` definida anteriormente.

* Por otro lado, se define el objeto que representa al modelo, lo cual se hace creando una subclase de `torch.nn.Module`.

* Finalmente se hace el entrenamiento y se verifican los resultados en el conjunto de validación. 